In [ ]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
from tqdm import tqdm
import pandas as pd
from glob import glob


In [ ]:
# Paths to directories
dam_dir = 'data/DAM'
test_dir = 'data/test_image_headmind'
extensions = ['*.jpg', '*.jpeg', '*.png']

# Get list of image file paths for DAM and Test
dam_images = glob(os.path.join(dam_dir, '*.jpeg'))
dam_images.sort()

test_images = []
for ext in extensions:
    pattern = os.path.join(test_dir, ext)
    test_images.extend(glob(pattern))
test_images.sort()

# Create DataFrames
dam_df = pd.DataFrame({'image_path': dam_images})
test_df = pd.DataFrame({'image_path': test_images})

print("DAM DataFrame:")
print(dam_df.head())

print("\nTest DataFrame:")
print(test_df.head())


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

image_path = "data/DAM/022P24A3328X4220.jpeg"  # Remplacez par le chemin de votre image
image = Image.open(image_path)
# image = image.resize((256, 256))  # Redimensionner pour uniformité
image_array = np.array(image)[np.newaxis, ...]

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='constant',
    cval=255
)

augmented_images = datagen.flow(image_array, batch_size=1)

for i in range(9):
    augmented_image = next(augmented_images)[0].astype('uint8')
    plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_image)
    plt.axis('off')



In [ ]:
from tqdm import tqdm
import os
import pandas as pd

# Chemin du dossier contenant les images mères
source_dir = "data/DAM"  # Dossier d'origine
target_dir = "data/augmented_dam"  # Dossier cible pour les images augmentées
os.makedirs(target_dir, exist_ok=True)

# Chemin pour le dossier des étiquettes
label_dir = "labels"
os.makedirs(label_dir, exist_ok=True)

# Initialiser un DataFrame pour enregistrer les classes
data_records = []

# Parcourir les images mères dans le dossier source avec tqdm
for image_name in tqdm(os.listdir(source_dir), desc="Processing images"):
    if image_name.endswith(('.jpg', '.jpeg', '.png')):  # Filtrer les fichiers d'image
        # Charger et préparer l'image
        image_path = os.path.join(source_dir, image_name)
        image = Image.open(image_path)
        # image = image.resize((256, 256))  # Redimensionner pour uniformité
        image_array = np.array(image)[np.newaxis, ...]  # Ajouter une dimension batch

        # Ajouter la classe mère dans les données
        data_records.append({"image_name": image_name, "class": image_name})

        # Créer des images augmentées
        datagen = ImageDataGenerator(
            rotation_range=30,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='constant',
            cval=255
        )

        augmented_images = datagen.flow(image_array, batch_size=1)

        # Sauvegarder 15 images augmentées
        for i in range(15):
            augmented_image = next(augmented_images)[0].astype('uint8')
            augmented_image_name = f"{os.path.splitext(image_name)[0]}_aug_{i + 1}.jpeg"
            augmented_image_path = os.path.join(target_dir, augmented_image_name)
            
            # Sauvegarder l'image augmentée
            augmented_image_pil = Image.fromarray(augmented_image)
            augmented_image_pil.save(augmented_image_path)

            # Ajouter la classe fille dans les données
            data_records.append({"image_name": augmented_image_name, "class": image_name})

        # Copier l'image mère dans le dossier cible
        image.save(os.path.join(target_dir, image_name))

# Sauvegarder les classes et noms d'images dans un fichier CSV
csv_path = os.path.join(label_dir, "image_classes.csv")
df = pd.DataFrame(data_records)
df["class"] = df["class"].apply(lambda x: os.path.splitext(x)[0])

# Sauvegarder le fichier CSV mis à jour
df.to_csv(csv_path, index=False)